In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, BatchNormalization, LeakyReLU
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import time


In [2]:
NAME = "single_rgb_image_regression_V01_{}".format(int(time.time()))


In [3]:
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [4]:
# def create_data_via_ImageDataGenerator():
#     train = ImageDataGenerator(rescale=1/255)
#     validation = ImageDataGenerator(rescale=1/255)
#     train_dataset = train.flow_from_directory("../Data/Train/smallTown",
#                                               target_size=(60, 60), batch_size=3, class_mode='binary')
#     validation_dataset = train.flow_from_directory("../Data/Train/smallTown",
#                                                target_size=(60, 60), batch_size=3, class_mode='binary')


In [5]:
def load_data_via_numpy_load():
    return (np.load("CNN_trainset_single_RGB_regression_V01_X.npy")/255, np.load("CNN_trainset_single_RGB_regression_V01_Y.npy"))


In [6]:
X, Y = load_data_via_numpy_load()


In [7]:
X


array([[[[0.18431373, 0.10980392, 0.00784314],
         [0.22745098, 0.15294118, 0.05098039],
         [0.23921569, 0.16470588, 0.0627451 ],
         ...,
         [0.21176471, 0.1254902 , 0.01960784],
         [0.22745098, 0.1372549 , 0.03529412],
         [0.23921569, 0.15294118, 0.05098039]],

        [[0.18823529, 0.11764706, 0.01568627],
         [0.18823529, 0.11372549, 0.01568627],
         [0.21568627, 0.14117647, 0.04313725],
         ...,
         [0.21960784, 0.13333333, 0.02745098],
         [0.20392157, 0.11764706, 0.01176471],
         [0.20392157, 0.11764706, 0.01176471]],

        [[0.18823529, 0.11372549, 0.01960784],
         [0.18039216, 0.10588235, 0.01176471],
         [0.17647059, 0.10588235, 0.00784314],
         ...,
         [0.21960784, 0.13333333, 0.02352941],
         [0.25098039, 0.16862745, 0.05882353],
         [0.25490196, 0.16862745, 0.05882353]],

        ...,

        [[0.19607843, 0.11764706, 0.01568627],
         [0.21960784, 0.14509804, 0.04313725]

In [8]:
X.shape

(6000, 60, 60, 3)

In [9]:


X = X.reshape(-1, 60, 60, 3)

X.shape


(6000, 60, 60, 3)

In [10]:
def build_model():
    model = Sequential()

    # 1st Conv-Layer
    model.add(Conv2D(128, (3, 3), input_shape=(X.shape[1:])))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2, 2))

    # 2nd Conv-Layer
    model.add(Conv2D(192, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    
    # 3rd Conv-Layer
    model.add(Conv2D(320, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # 4th Conv-Layer
    model.add(Conv2D(576, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())

    # 1st Fully-Connected
    model.add(Dense(4096))
    model.add(Activation(LeakyReLU(alpha=0.2)))

    # 2nd Fully-Connected
    model.add(Dense(4096))
    model.add(Activation(LeakyReLU(alpha=0.2)))

    # 3rd Fully-Connected
    model.add(Dense(1))
    model.add(Activation("sigmoid"))
    return model


model = build_model()


In [11]:

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 58, 58, 128)       3584      
_________________________________________________________________
activation (Activation)      (None, 58, 58, 128)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 58, 58, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 192)       221376    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 192)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 192)       7

In [12]:
model.fit(X, Y, batch_size=32, epochs=1, validation_split=0.1, callbacks=[tensorboard])

 20/169 [==>...........................] - ETA: 2:00 - loss: 0.1427 - accuracy: 0.3391

KeyboardInterrupt: 

# Test


In [ ]:
# model = tf.keras.models.load_model("./models/single_rgb_image_big_dataset")


In [ ]:
# # Alias

# img1 = Image.open(
#     "D:\Main\MA_PROGR\Data\Train\LED_Wand_Aufnahmen\Alias\LED_Wand_20001.png")
# img1 = np.asarray(img1)/255




# img1 = img1.reshape(1,60,60,3)
# print(img1.shape)
# # print(img1)
# res1 = model.predict(img1)
# print(res1)


In [ ]:
# # NoAlias

# img2 = Image.open(
#     "D:\\Main\\MA_PROGR\\Data\\Train\\LED_Wand_Aufnahmen\\NoAlias\\abstract_cross_203.png")
# img2 = np.asarray(img2)/255

# img2 = img2.reshape(1, 60, 60, 3)
# img2.shape
# # print(img2)
# res2 = model.predict(img2)
# print(res2)


In [ ]:
# import matplotlib.pyplot as plt
# plt.imshow(img2.reshape(60, 60, 3))


In [ ]:
tf.keras.models.save_model(model, "./models/{}".format(NAME))

INFO:tensorflow:Assets written to: ./models/single_rgb_image_trainset_V04_1653260543\assets


INFO:tensorflow:Assets written to: ./models/single_rgb_image_trainset_V04_1653260543\assets
